In [ ]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor
### google drive
from google.colab import drive
#import gspread
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install --ignore-installed sddk ### our own package under construction, always install to have up-to-date version
import sddk

     |████████████████████████████████| 63.8MB 55kB/s 
     |████████████████████████████████| 10.1MB 45.5MB/s 
     |████████████████████████████████| 12.4MB 46.5MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 14.6MB 321kB/s 
     |████████████████████████████████| 235kB 49.3MB/s 
     |████████████████████████████████| 512kB 36.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 133kB 54.3MB/s 
     |████████████████████████████████| 143kB 51.3MB/s 
     |████████████████████████████████| 163kB 51.0MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.9 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.1

In [ ]:
### configure session and groupurl
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


### SDDK usage

Most of the script below has been written an older version of the sddk passage. Since version 1.7, you mostly do not need commands like `s.put()` or `s.get` with a complex parametrization for writing or reading json files etc. You can just use `sddk.read_file()` and `sddk.write_file()` following th [documentation](https://github.com/sdam-au/sddk_py).


# Geographies on the province basis

In [ ]:
### extract names of provinces
response = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/terms/province")
response
json_data = response.json()
print(str(json_data)[:200])

{'provinces': {'Ach': 'Achaia', 'Aeg': 'Aegyptus', 'Aem': 'Aemilia (Regio VIII)', 'Afr': 'Africa Proconsularis', 'AlC': 'Alpes Cottiae', 'AlG': 'Alpes Graiae', 'AlM': 'Alpes Maritimae', 'AlP': 'Alpes 


In [ ]:
###  generate a list of provinces, resp. their abbreviations
%%time

json_data = requests.get("https://edh-www.adw.uni-heidelberg.de/data/api/terms/province").json()
provinces = json_data["provinces"].keys()

### make the basic URL form
URL_form_geo = "https://edh-www.adw.uni-heidelberg.de/data/api/geography/search?"

### parse all find spot for each province 
### at them to the list
geo_data = []
for province in provinces:
  total = requests.get(URL_form_geo + "province=" + province).json()["total"]
  response = requests.get(URL_form_geo + "province=" + province + "&limit=" + str(total))
  geo_data.extend(response.json()["items"])
  print(province, total)

Ach 183
Aeg 65
Aem 162
Afr 1400
AlC 59
AlG 31
AlM 173
AlP 79
ApC 204
Aqu 212
Ara 103
Arm 2
Asi 182
Ass 0
Bae 1305
Bar 89
Bel 429
BiP 105
BrL 79
Bri 1950
Cap 31
Cil 48
Cor 15
Cre 29
Cyp 21
Cyr 24
Dac 996
Dal 2194
Epi 77
Etr 356
Gal 89
GeI 1064
GeS 2661
HiC 2553
Inc 7
Iud 83
LaC 654
Lig 131
Lug 254
Lus 746
LyP 37
MaC 298
MaT 43
Mak 434
Mes 9
MoI 888
MoS 790
Nar 635
Nor 1504
Num 333
PaI 1316
PaS 1623
Pic 85
Rae 729
ReB 22
Rom 734
Sam 313
Sar 81
Sic 47
Syr 154
Thr 257
Tra 111
Tri 0
Umb 212
Val 0
VeH 479
CPU times: user 1.75 s, sys: 143 ms, total: 1.89 s
Wall time: 2min 19s


In [ ]:
### total amount of findspots
len(geo_data)

29979

In [ ]:
geo_data_df = pd.DataFrame(geo_data)
geo_data_df.set_index("uri", inplace=True)
geo_data_df.head(5)

,country,province,last_update,id,find_spot_ancient,coordinates,region,find_spot_modern,find_spot,pleiades_uri,geonames_uri
uri,,,,,,,,,,,
https://edh-www.adw.uni-heidelberg.de/edh/geographie/10,Greece,Achaia,2011-07-14,10,Ptoion,"37.05,25.19",NaN,NaN,NaN,NaN,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/100,Greece,Achaia,2012-03-20,100,Athenae,"37.983175,23.716647",Attikí,Athínai,Kerameikos,https://pleiades.stoa.org/places/579885,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/101,Greece,Achaia,2012-03-20,101,Athenae,"37.983175,23.716647",Attikí,Athínai,Kerameikos bei Kirche Haghia Triada,https://pleiades.stoa.org/places/579885,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/102,Greece,Achaia,2012-03-20,102,Colonia Augusta Aroe Patrae,"38.251123,21.741943",Dytikí Elláda,Pátrai,"Kirche, sekundär verwendet",NaN,NaN
https://edh-www.adw.uni-heidelberg.de/edh/geographie/103,Greece,Achaia,2012-03-20,103,Colonia Laus Iulia Corinthus,"37.939865,22.928467",Pelopónissos,Kórinthos,Kranion,NaN,NaN


## Upload the data to sciencedata.dk

In [ ]:
### put your dataframe data into this folder
s.put(sddk_url + "SDAM_data/EDH/EDH_geographies_raw.json", data=geo_data_df.to_json())

<Response [201]>

# Merge the inscriptions with geographies

In [ ]:
### import the geographies
geo_data_df = sddk.read_file("SDAM_data/EDH/EDH_geographies_raw.json", "df", conf)
geo_data_df.head(5)

,country,last_update,coordinates,province,id,find_spot_ancient,pleiades_uri,find_spot,region,find_spot_modern,geonames_uri
https://edh-www.adw.uni-heidelberg.de/edh/geographie/10,Greece,2011-07-14,"37.05,25.19",Achaia,10,Ptoion,None,None,None,None,None
https://edh-www.adw.uni-heidelberg.de/edh/geographie/100,Greece,2012-03-20,"37.983175,23.716647",Achaia,100,Athenae,https://pleiades.stoa.org/places/579885,Kerameikos,Attikí,Athínai,None
https://edh-www.adw.uni-heidelberg.de/edh/geographie/101,Greece,2012-03-20,"37.983175,23.716647",Achaia,101,Athenae,https://pleiades.stoa.org/places/579885,Kerameikos bei Kirche Haghia Triada,Attikí,Athínai,None
https://edh-www.adw.uni-heidelberg.de/edh/geographie/102,Greece,2012-03-20,"38.251123,21.741943",Achaia,102,Colonia Augusta Aroe Patrae,None,"Kirche, sekundär verwendet",Dytikí Elláda,Pátrai,None
https://edh-www.adw.uni-heidelberg.de/edh/geographie/103,Greece,2012-03-20,"37.939865,22.928467",Achaia,103,Colonia Laus Iulia Corinthus,None,Kranion,Pelopónissos,Kórinthos,None


In [ ]:
### upload the raw inscriptions (extracted in another notebook)
### use the line below to upload the old data (obtained on province basis)
### inscriptions_data_df = pd.DataFrame(s.get("https://sciencedata.dk/files/personal_folder/EDH_data/EDH_inscriptions_raw.json").json())
inscriptions_data_df = sddk.read_file("SDAM_data/EDH/EDH_onebyone.json", "df", conf)
inscriptions_data_df.head(5)

,diplomatic_text,literature,trismegistos_uri,id,findspot_ancient,not_before,type_of_inscription,work_status,edh_geography_uri,not_after,country,province_label,transcription,material,height,width,findspot_modern,depth,commentary,uri,responsible_individual,last_update,language,modern_region,letter_size,type_of_monument,people,year_of_find,findspot,present_location,external_image_uris,religion,fotos,geography,military,social_economic_legal_history
0,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",https://www.trismegistos.org/text/251193,HD000001,"Cumae, bei",0071,epitaph,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0130,Italy,Latium et Campania (Regio I),D(is) M(anibus) / Noniae P(ubli) f(iliae) Opta...,"Marmor, geädert / farbig",33 cm,34 cm,"Cuma, bei",2.7 cm,(C): 2. Hälfte 1. - Anfang 2. Jh. - AE; Ende ...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Campania,3.2-2 cm,tabula,"[{'cognomen': 'Optata', 'person_id': '1', 'gen...",None,None,None,None,None,None,None,None,None
1,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265631,HD000002,Roma,0051,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,C(aius) Sextius Paris / qui vixit / annis LXX,marble: rocks - metamorphic rocks,28 cm,85 cm,Roma,None,AE 1983: Breite: 35 cm.,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Lazio,4 cm,tabula,"[{'age: years': '70', 'cognomen': 'Paris', 'ge...",1937,"Via Nomentana, S. Alessandro, Kirche",None,None,None,None,None,None,None
2,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",https://www.trismegistos.org/text/220675,HD000003,None,0131,honorific inscription,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0170,Spain,Baetica,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,marble: rocks - metamorphic rocks,(37) cm,(34) cm,Tomares,(12) cm,(B): [S]isenna ist falscher Kasus; folgende E...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2006-08-31,Latin,Sevilla,4.5-3 cm,statue base,"[{'nomen': 'Mummius+', 'cognomen': 'Sisenna+ R...",before 1975,None,"Sevilla, Privatbesitz",None,None,None,None,None,None
3,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",https://www.trismegistos.org/text/222102,HD000004,Ipolcobulcula,0151,votive inscription,checked with photo,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Spain,Baetica,[---?]AV(?)S(?)[---]L(?)L(?)A / M(arci) Porci ...,limestone: rocks - clastic sediments,(39) cm,27 cm,Carcabuey,18 cm,Material: lokaler grauer Kalkstein. (B): Styl...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Gräf,2015-03-27,Latin,Córdoba,2.5 cm,altar,"[{'cognomen': '[---]', 'status': 'slaves', 'pe...",before 1979,None,"Carcabuey, Grupo Escolar",[http://cil-old.bbaw.de/test06/bilder/datenban...,names of pagan deities,None,None,None,None
4,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265629,HD000005,Roma,0001,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,[---] l(ibertus) Successus / [---] L(uci) l(ib...,None,None,None,Roma,None,(B): Z. 3: C(ai) l(ibertae) Tyches.,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2010-01-04,Latin,Lazio,None,stele,"[{'status': 'freedmen / freedwomen', 'name': '...",None,Via Cupa (ehem. Vigna Nardi),None,None,None,None,None,None,None


In [ ]:
### use "edh_geography_uri" column to
### put information from geo_data_df "coordinates" column
### and return it as a new "coordinates" column in the inscriptions_data_df 

%%time
def get_coordinates(column):
  try:
    return geo_data_df.loc[column]["coordinates"]
  except:
    return None
inscriptions_data_df["coordinates"]  =  inscriptions_data_df.apply(lambda row: get_coordinates(row["edh_geography_uri"]), axis=1)

CPU times: user 10.9 s, sys: 2.83 ms, total: 10.9 s
Wall time: 10.9 s


In [ ]:
len(inscriptions_data_df)

80270

In [ ]:
len(inscriptions_data_df[inscriptions_data_df["coordinates"].notnull()]) ### in the previous version (on province basis), we were able to geolocate 69735 from 72483 inscriptions

77296

# Merge with data from XML files

In [ ]:
EDH_xml_data = sddk.read_file("SDAM_data/EDH/edh_xml_data_2020-06-23.json", "df", conf)
EDH_xml_data.head(5)

,idno_uri,idno_tm,placenames_refs,text,origdate_text,origdate_attrs,objecttype,commentary
0,http://edh-www.adw.uni-heidelberg.de/edh/insch...,214091,"[http://www.trismegistos.org/place/019860, htt...",In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"{'notbefore-custom': '0551', 'notafter-custom'...","[Tafel, 257]",\n\n
1,http://edh-www.adw.uni-heidelberg.de/edh/insch...,415668,"[http://www.trismegistos.org/place/016530, htt...",Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"{'notbefore-custom': '0071', 'notafter-custom'...","[Tafel, 257]",\n\n
2,http://edh-www.adw.uni-heidelberg.de/edh/insch...,192497,"[http://www.trismegistos.org/place/029481, htt...",Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"{'notbefore-custom': '0197', 'datingmethod': '...","[unbestimmt, 2]",\n Textwiedergabe nach Eck.\n
3,http://edh-www.adw.uni-heidelberg.de/edh/insch...,199108,"[http://www.trismegistos.org/place/015771, htt...",Saturno Meddensi Augusto sacru...,,{'datingmethod': 'http://en.wikipedia.org/wiki...,"[unbestimmt, 2]",\n\n
4,http://edh-www.adw.uni-heidelberg.de/edh/insch...,201034,"[http://www.trismegistos.org/place/015771, htt...",Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"{'notbefore-custom': '0193', 'datingmethod': '...","[unbestimmt, 2]",\n Datierung: L. Naevius Quadratianus war 193 ...


In [ ]:
EDH_xml_data = EDH_xml_data[["idno_uri", "text", "origdate_text", "origdate_attrs", "objecttype"]] ### only subselection
EDH_xml_data.columns = ["uri", "text_cleaned", "origdate_text", "origdate_attrs","objecttype"] ### new columns, useful for merging

In [ ]:
len(EDH_xml_data)

78631

In [ ]:
EDH_xml_data["uri"].replace({r"http:" : r"https:"}, regex=True, inplace=True) ### URIs via API follow HTTPS protocol
EDH_xml_data.head(5)

,uri,text_cleaned,origdate_text,origdate_attrs,objecttype
0,https://edh-www.adw.uni-heidelberg.de/edh/insc...,In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"{'notbefore-custom': '0551', 'notafter-custom'...","[Tafel, 257]"
1,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"{'notbefore-custom': '0071', 'notafter-custom'...","[Tafel, 257]"
2,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"{'notbefore-custom': '0197', 'datingmethod': '...","[unbestimmt, 2]"
3,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Saturno Meddensi Augusto sacru...,,{'datingmethod': 'http://en.wikipedia.org/wiki...,"[unbestimmt, 2]"
4,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"{'notbefore-custom': '0193', 'datingmethod': '...","[unbestimmt, 2]"


In [ ]:
inscriptions_data_df.shape

(80270, 37)

In [ ]:
inscriptions_rich = inscriptions_data_df.merge(EDH_xml_data, on="uri", how="left")
inscriptions_rich.head(5)

,diplomatic_text,literature,trismegistos_uri,id,findspot_ancient,not_before,type_of_inscription,work_status,edh_geography_uri,not_after,country,province_label,transcription,material,height,width,findspot_modern,depth,commentary,uri,responsible_individual,last_update,language,modern_region,letter_size,type_of_monument,people,year_of_find,findspot,present_location,external_image_uris,religion,fotos,geography,military,social_economic_legal_history,coordinates,text_cleaned,origdate_text,origdate_attrs,objecttype
0,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",https://www.trismegistos.org/text/251193,HD000001,"Cumae, bei",0071,epitaph,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0130,Italy,Latium et Campania (Regio I),D(is) M(anibus) / Noniae P(ubli) f(iliae) Opta...,"Marmor, geädert / farbig",33 cm,34 cm,"Cuma, bei",2.7 cm,(C): 2. Hälfte 1. - Anfang 2. Jh. - AE; Ende ...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Campania,3.2-2 cm,tabula,"[{'cognomen': 'Optata', 'person_id': '1', 'gen...",None,None,None,None,None,None,None,None,None,"40.8471577,14.0550756",Dis Manibus Noniae Publi filiae Optatae et Cai...,71 AD – 130 AD,"{'notbefore-custom': '0071', 'notafter-custom'...","[Tafel, 257]"
1,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265631,HD000002,Roma,0051,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,C(aius) Sextius Paris / qui vixit / annis LXX,marble: rocks - metamorphic rocks,28 cm,85 cm,Roma,None,AE 1983: Breite: 35 cm.,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Lazio,4 cm,tabula,"[{'age: years': '70', 'cognomen': 'Paris', 'ge...",1937,"Via Nomentana, S. Alessandro, Kirche",None,None,None,None,None,None,None,"41.895466,12.482324",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,"{'notbefore-custom': '0051', 'notafter-custom'...","[Tafel, 257]"
2,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",https://www.trismegistos.org/text/220675,HD000003,None,0131,honorific inscription,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0170,Spain,Baetica,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,marble: rocks - metamorphic rocks,(37) cm,(34) cm,Tomares,(12) cm,(B): [S]isenna ist falscher Kasus; folgende E...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2006-08-31,Latin,Sevilla,4.5-3 cm,statue base,"[{'nomen': 'Mummius+', 'cognomen': 'Sisenna+ R...",before 1975,None,"Sevilla, Privatbesitz",None,None,None,None,None,None,"37.37281,-6.04589",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,"{'notbefore-custom': '0131', 'notafter-custom'...","[Statuenbasis, 57]"
3,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",https://www.trismegistos.org/text/222102,HD000004,Ipolcobulcula,0151,votive inscription,checked with photo,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Spain,Baetica,[---?]AV(?)S(?)[---]L(?)L(?)A / M(arci) Porci ...,limestone: rocks - clastic sediments,(39) cm,27 cm,Carcabuey,18 cm,Material: lokaler grauer Kalkstein. (B): Styl...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Gräf,2015-03-27,Latin,Córdoba,2.5 cm,altar,"[{'cognomen': '[---]', 'status': 'slaves', 'pe...",before 1979,None,"Carcabuey, Grupo Escolar",[http://cil-old.bbaw.de/test06/bilder/datenban...,names of pagan deities,None,None,None,None,"37.4442,-4.27471",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,"{'notbefore-custom': '0151', 'notafter-custom'...","[Altar, 29]"
4,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265629,HD000005,Roma,0001,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,[---] l(ibertus

In [ ]:
inscriptions_rich.shape

(80270, 41)

# Upload the merged data to sciencedata.dk

In [ ]:
### put your dataframe data into this folder
%%time
sddk.write_file("SDAM_data/EDH/EDH_merged_2020-06-23.json")
#s.put(sddk_url + "SDAM_data/EDH/EDH_inscriptions_rich.json", data=inscriptions_rich.to_json())

CPU times: user 1.17 s, sys: 220 ms, total: 1.39 s
Wall time: 12.7 s


<Response [204]>

# Try a version with different encoding (backup)


In [ ]:
inscriptions_rich = sddk.read_file("SDAM_data/EDH/EDH_inscriptions_rich.json", "df", auth)

In [ ]:
inscriptions_rich.shape

(80270, 40)

In [ ]:
auth[0].put(auth[1] + "SDAM_data/EDH/EDH_utf8.json", data=inscriptions_rich.to_json(force_ascii=False).encode("utf-8"))

<Response [204]>

In [ ]:
EDH_utf8_sample = inscriptions_rich[:1000]


In [ ]:
EDH_utf8_sample.columns[15]

'width'

In [ ]:
EDH_utf8_sample.tail(6)["height"]

994       None
995     172 cm
996       None
997      68 cm
998    (65) cm
999    (73) cm
Name: height, dtype: object

In [ ]:
### a sample for testing
auth[0].put(auth[1] + "SDAM_data/EDH/EDH_utf8_sample.json", data=inscriptions_rich[:1000].to_json(force_ascii=False).encode("utf-8"))

<Response [201]>

In [ ]:
### testing whether I am able to get it back
EDH_utf8 = sddk.read_file("SDAM_data/EDH/EDH_utf8.json", "df", auth)

In [ ]:
EDH_utf8.head(5)

,diplomatic_text,literature,trismegistos_uri,id,findspot_ancient,not_before,type_of_inscription,work_status,edh_geography_uri,not_after,country,province_label,transcription,material,height,width,findspot_modern,depth,commentary,uri,responsible_individual,last_update,language,modern_region,letter_size,type_of_monument,people,year_of_find,findspot,present_location,external_image_uris,religion,fotos,geography,military,social_economic_legal_history,coordinates,text_cleaned,origdate_text,objecttype
0,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",https://www.trismegistos.org/text/251193,HD000001,"Cumae, bei",0071,epitaph,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0130,Italy,Latium et Campania (Regio I),D(is) M(anibus) / Noniae P(ubli) f(iliae) Opta...,"Marmor, geädert / farbig",33 cm,34 cm,"Cuma, bei",2.7 cm,(C): 2. Hälfte 1. - Anfang 2. Jh. - AE; Ende ...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Campania,3.2-2 cm,tabula,"[{'cognomen': 'Optata', 'person_id': '1', 'gen...",None,None,None,None,None,None,None,None,None,"40.8471577,14.0550756",Dis Manibus Noniae Publi filiae Optatae et Cai...,71 AD – 130 AD,"[Tafel, 257]"
1,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265631,HD000002,Roma,0051,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,C(aius) Sextius Paris / qui vixit / annis LXX,marble: rocks - metamorphic rocks,28 cm,85 cm,Roma,None,AE 1983: Breite: 35 cm.,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2014-04-07,Latin,Lazio,4 cm,tabula,"[{'age: years': '70', 'cognomen': 'Paris', 'ge...",1937,"Via Nomentana, S. Alessandro, Kirche",None,None,None,None,None,None,None,"41.895466,12.482324",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,"[Tafel, 257]"
2,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",https://www.trismegistos.org/text/220675,HD000003,None,0131,honorific inscription,provisional,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0170,Spain,Baetica,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,marble: rocks - metamorphic rocks,(37) cm,(34) cm,Tomares,(12) cm,(B): [S]isenna ist falscher Kasus; folgende E...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2006-08-31,Latin,Sevilla,4.5-3 cm,statue base,"[{'nomen': 'Mummius+', 'cognomen': 'Sisenna+ R...",before 1975,None,"Sevilla, Privatbesitz",None,None,None,None,None,None,"37.37281,-6.04589",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,"[Statuenbasis, 57]"
3,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",https://www.trismegistos.org/text/222102,HD000004,Ipolcobulcula,0151,votive inscription,checked with photo,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Spain,Baetica,[---?]AV(?)S(?)[---]L(?)L(?)A / M(arci) Porci ...,limestone: rocks - clastic sediments,(39) cm,27 cm,Carcabuey,18 cm,Material: lokaler grauer Kalkstein. (B): Styl...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Gräf,2015-03-27,Latin,Córdoba,2.5 cm,altar,"[{'cognomen': '[---]', 'status': 'slaves', 'pe...",before 1979,None,"Carcabuey, Grupo Escolar",[http://cil-old.bbaw.de/test06/bilder/datenban...,names of pagan deities,None,None,None,None,"37.4442,-4.27471",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,"[Altar, 29]"
4,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",https://www.trismegistos.org/text/265629,HD000005,Roma,0001,epitaph,no image,https://edh-www.adw.uni-heidelberg.de/edh/geog...,0200,Italy,Roma,[---] l(ibertus) Successus / [---] L(uci) l(ib...,None,None,None,Roma,None,(B): Z. 3: C(ai) l(ibertae) Tyches.,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Feraudi,2010-01-04,Latin,Lazio,None,stele,"[{'status': 'freedmen / freedwomen